# Proyecto Final - Ciencia de datos en Python
#### Ingeniería de datos con Python
Jerson Ochoa - 15004111

### Scope del proyecto: 
Consiste en la elaboración de un pipeline de ingeniería de datos para la construcción de un DataWarehouse con base en el **análisis de la ejecución presupuestaria de los 340 municipios de Guatemala de los años 2019, 2020 y 2021**
Las fuentes de datos provienen del archivo de información del **Ministerio de Finanzas Guatemala** publicado en https://datos.minfin.gob.gt/. 
El objetivo del proyecto es analizar y comparar la ejecución presupuestaria (ingresos y egresos) de cada uno de los municipios  históricamente con los datos que provee el ministerio, la idea es preparar el pipeline para ingresar los datos conforme el ministerio los publica debido a que actualmente (año 2023) solo existen datos del 2021 para atrás. 
Los archivos publicados por el Ministerio de Finanzas se encuentran en formato CSV, se almacenarán en una base de datos RDS, se unificarán, se limpiarán y se transformarán los datos por medio de funciones de Python para formar un programa que realice el proceso, luego se creará una BD en formato estrella para completar el DataWarehouse. 

#### Distribución de las fuentes
- Se tienen 3 archivos csv, por lo tanto se cargará uno a una BD en RDS y los otros dos se cargarán a un bucket en S3 para cumplir con los requerimientos del proyecto

## Preguntas que se intentarán responder con el análisis de datos
1. ¿Cuál es el top 10 de municipios con el mayor ingreso global en 2021?
2. ¿Cuál es el top 10 de municipios con el menor ingreso global en 2021?
3. ¿Cuál es el top 10 de municipios con el mayor ingreso por tributos en 2021?
4. ¿Cuál es el top 10 de municipios con el mayor reajuste de presupuesto en 2021?
5. ¿Cuál es el top 10 de municipios con el mayor gasto en arrendamientos en 2021?

#### Nota: Las preguntas de análisis se hacen en base al año 2021, pero perfectamente deben poder ajustarse a cualquier año con datos cargados.

### Carga de librerías

In [14]:
import pandas as pd
import boto3
import psycopg2
import datetime
import numpy as np
import configparser
import mysql.connector as mysqlC

### Carga de datos y exploración

In [2]:
# Datos 2019
datos_2019 = pd.read_csv('datasets/ingresos-municipales-2019.csv', sep=",", encoding='utf-8')
pd.set_option('display.max_columns', None)
datos_2019.head()

,Ejercicio,REGION,Código Entidad,Entidad,Código Departamento,Departamento,Código Clase,Clase,Código Sección,Sección,Código Grupo,Grupo,Código Recurso,Recurso,Asignado,Modificado,Vigente,Devengado,Percibido
0,2019,REGIÓN I METROPOLITANA,12100101,MUNICIPALIDAD DE GUATEMALA,100,GUATEMALA,10000,INGRESOS TRIBUTARIOS,10100,IMPUESTOS DIRECTOS,10140,IMPUESTOS MUNICIPALES,10141,SOBRE LA TENENCIA DE PATRIMONIO,532810000.0,17877000.0,550687000.0,5.127175e+08,5.127175e+08
1,2019,REGIÓN I METROPOLITANA,12100101,MUNICIPALIDAD DE GUATEMALA,100,GUATEMALA,10000,INGRESOS TRIBUTARIOS,10200,IMPUESTOS INDIRECTOS,10220,IMPUESTOS SOBRE PRODUCTOS INDUSTRIALES Y PRIMA...,10223,SOBRE DERIVADOS DEL PETRÓLEO,75000000.0,0.0,75000000.0,5.809833e+07,5.809833e+07
2,2019,REGIÓN I METROPOLITANA,12100101,MUNICIPALIDAD DE GUATEMALA,100,GUATEMALA,10000,INGRESOS TRIBUTARIOS,10200,IMPUESTOS INDIRECTOS,10220,IMPUESTOS SOBRE PRODUCTOS INDUSTRIALES Y PRIMA...,10224,REGALÍAS,1200000.0,180000.0,1380000.0,1.001869e+06,1.001869e+06
3,2019,REGIÓN I METROPOLITANA,12100101,MUNICIPALIDAD DE GUATEMALA,100,GUATEMALA,10000,INGRESOS TRIBUTARIOS,10200,IMPUESTOS INDIRECTOS,10280,ARBITRIOS MUNICIPALES,10282,SOBRE ESTABLECIMIENTOS DE SERVICIOS,57000.0,1000.0,58000.0,3.537000e+04,3.537000e+04
4,2019,REGIÓN I METROPOLITANA,12100101,MUNICIPALIDAD DE GUATEMALA,100,GUATEMALA,10000,INGRESOS TRIBUTARIOS,10200,IMPUESTOS INDIRECTOS,10280,ARBITRIOS MUNICIPALES,10284,SOBRE DIVERSIONES Y ESPECTÁCULOS,3078000.0,460000.0,3538000.0,2.956961e+06,2.956961e+06


In [3]:
# Datos 2020
datos_2020 = pd.read_csv('datasets/ingresos-municipales-2020.csv', sep=",", encoding='utf-8')
pd.set_option('display.max_columns', None)
datos_2020.head()

,Ejercicio,REGION,Código Entidad,Entidad,Código Departamento,Departamento,Código Clase,Clase,Código Sección,Sección,Código Grupo,Grupo,Código Recurso,Recurso,Asignado,Modificado,Vigente,Devengado,Percibido
0,2020,REGIÓN I METROPOLITANA,12100101,MUNICIPALIDAD DE GUATEMALA,100,GUATEMALA,10000,INGRESOS TRIBUTARIOS,10100,IMPUESTOS DIRECTOS,10140,IMPUESTOS MUNICIPALES,10141,SOBRE LA TENENCIA DE PATRIMONIO,526223000.0,0.0,526223000.0,2.580033e+08,2.580033e+08
1,2020,REGIÓN I METROPOLITANA,12100101,MUNICIPALIDAD DE GUATEMALA,100,GUATEMALA,10000,INGRESOS TRIBUTARIOS,10200,IMPUESTOS INDIRECTOS,10220,IMPUESTOS SOBRE PRODUCTOS INDUSTRIALES Y PRIMA...,10223,SOBRE DERIVADOS DEL PETRÓLEO,70000000.0,0.0,70000000.0,3.826994e+07,3.826994e+07
2,2020,REGIÓN I METROPOLITANA,12100101,MUNICIPALIDAD DE GUATEMALA,100,GUATEMALA,10000,INGRESOS TRIBUTARIOS,10200,IMPUESTOS INDIRECTOS,10220,IMPUESTOS SOBRE PRODUCTOS INDUSTRIALES Y PRIMA...,10224,REGALÍAS,1770000.0,0.0,1770000.0,4.263171e+05,4.263171e+05
3,2020,REGIÓN I METROPOLITANA,12100101,MUNICIPALIDAD DE GUATEMALA,100,GUATEMALA,10000,INGRESOS TRIBUTARIOS,10200,IMPUESTOS INDIRECTOS,10280,ARBITRIOS MUNICIPALES,10282,SOBRE ESTABLECIMIENTOS DE SERVICIOS,57000.0,0.0,57000.0,2.153500e+04,2.153500e+04
4,2020,REGIÓN I METROPOLITANA,12100101,MUNICIPALIDAD DE GUATEMALA,100,GUATEMALA,10000,INGRESOS TRIBUTARIOS,10200,IMPUESTOS INDIRECTOS,10280,ARBITRIOS MUNICIPALES,10284,SOBRE DIVERSIONES Y ESPECTÁCULOS,4230000.0,0.0,4230000.0,1.084353e+06,1.084353e+06


In [4]:
# Datos 2021
datos_2021 = pd.read_csv('datasets/ejecucion-presupuestaria-de-ingresos-dafim.csv', sep='\t', encoding='latin-1')
pd.set_option('display.max_columns', None)
datos_2021.head()

,Identificador,Ejercicio,Mes,Region,CodigoEntidad,Entidad,CodigoDepartamento,Departamento,CodigoClase,Clase,CodigoSeccion,Seccion,CodigoGrupo,Grupo,CodigoRecurso,Recurso,Asignado,Modificado,Vigente,Devengado,Percibido
0,1,2021,3,REGIÓN VII NOROCCIDENTE,12101407,MUNICIPALIDAD DE PATZITE,1400,QUICHE,16000,TRANSFERENCIAS CORRIENTES,16200,DEL SECTOR PÚBLICO,16210,DE LA ADMINISTRACIÓN CENTRAL,16210,DE LA ADMINISTRACIÓN CENTRAL,0.0,0.0,0.0,162969.26,162969.26
1,2,2021,3,REGIÓN VII NOROCCIDENTE,12101407,MUNICIPALIDAD DE PATZITE,1400,QUICHE,10000,INGRESOS TRIBUTARIOS,10200,IMPUESTOS INDIRECTOS,10280,ARBITRIOS MUNICIPALES,10289,OTROS ARBITRIOS MUNICIPALES,0.0,0.0,0.0,3510.00,3510.00
2,3,2021,3,REGIÓN VII NOROCCIDENTE,12101407,MUNICIPALIDAD DE PATZITE,1400,QUICHE,10000,INGRESOS TRIBUTARIOS,10200,IMPUESTOS INDIRECTOS,10280,ARBITRIOS MUNICIPALES,10284,SOBRE DIVERSIONES Y ESPECTÁCULOS,0.0,0.0,0.0,180.00,180.00
3,4,2021,3,REGIÓN VII NOROCCIDENTE,12101407,MUNICIPALIDAD DE PATZITE,1400,QUICHE,11000,INGRESOS NO TRIBUTARIOS,11200,TASAS,11290,TASAS Y LICENCIAS VARIAS,11290,TASAS Y LICENCIAS VARIAS,0.0,0.0,0.0,29591.80,29591.80
4,5,2021,3,REGIÓN VII NOROCCIDENTE,12101407,MUNICIPALIDAD DE PATZITE,1400,QUICHE,11000,INGRESOS NO TRIBUTARIOS,11400,"ARRENDAMIENTO DE EDIFICIOS, EQUIPOS E INSTALAC...",11410,DE EDIFICIOS Y VIVIENDAS,11410,DE EDIFICIOS Y VIVIENDAS,0.0,0.0,0.0,180.00,180.00


### Cargar datos de 2021 a una BD en RDS

In [5]:
config = configparser.ConfigParser()
config.read('escec.cfg')

['escec.cfg']

#### Creamos instancia de RDS

In [6]:
aws_conn = boto3.client('rds', aws_access_key_id=config.get('IAM', 'ACCESS_KEY'),
                    aws_secret_access_key=config.get('IAM', 'SECRET_ACCESS_KEY'),
                    region_name='us-east-2')

#### Verificamos instancias de RDS disponibles

In [7]:
rdsInstanceIds = []

response = aws_conn.describe_db_instances()
for resp in response['DBInstances']:
    rdsInstanceIds.append(resp['DBInstanceIdentifier'])
    db_instance_status = resp['DBInstanceStatus']

print(f"DBInstanceIds {rdsInstanceIds}")

DBInstanceIds ['dw-db']


#### Creación de servicio RDS

In [8]:
rdsIdentifier = 'fuenteIngresosMuniMysql' 

In [9]:
try:
    response = aws_conn.create_db_instance(
            AllocatedStorage=10,
            DBName=config.get('RDS_MYSQL_F', 'DB_NAME'),
            DBInstanceIdentifier=rdsIdentifier,
            DBInstanceClass="db.t3.micro",
            Engine="mysql",
            MasterUsername=config.get('RDS_MYSQL_F', 'DB_USER'),
            MasterUserPassword=config.get('RDS_MYSQL_F', 'DB_PASSWORD'),
            Port=int(config.get('RDS_MYSQL_F', 'DB_PORT')),
            VpcSecurityGroupIds=[config.get('VPC', 'SECURITY_GROUP')],
            PubliclyAccessible=True
        )
    print(response)
except aws_conn.exceptions.DBInstanceAlreadyExistsFault as ex:
    print("La Instancia de Base de Datos ya Existe.")

{'DBInstance': {'DBInstanceIdentifier': 'fuenteingresosmunimysql', 'DBInstanceClass': 'db.t3.micro', 'Engine': 'mysql', 'DBInstanceStatus': 'creating', 'MasterUsername': 'root', 'DBName': 'fuente_ingresos_muni_mysql', 'AllocatedStorage': 10, 'PreferredBackupWindow': '09:38-10:08', 'BackupRetentionPeriod': 1, 'DBSecurityGroups': [], 'VpcSecurityGroups': [{'VpcSecurityGroupId': 'sg-08fdb378f31ed0d0f', 'Status': 'active'}], 'DBParameterGroups': [{'DBParameterGroupName': 'default.mysql8.0', 'ParameterApplyStatus': 'in-sync'}], 'DBSubnetGroup': {'DBSubnetGroupName': 'default', 'DBSubnetGroupDescription': 'default', 'VpcId': 'vpc-09e2d36e7f45d1f52', 'SubnetGroupStatus': 'Complete', 'Subnets': [{'SubnetIdentifier': 'subnet-0b304952bbcd0bc46', 'SubnetAvailabilityZone': {'Name': 'us-east-2b'}, 'SubnetOutpost': {}, 'SubnetStatus': 'Active'}, {'SubnetIdentifier': 'subnet-0b52320ca9b4e282e', 'SubnetAvailabilityZone': {'Name': 'us-east-2c'}, 'SubnetOutpost': {}, 'SubnetStatus': 'Active'}, {'SubnetI

#### Obtenemos url del host

In [10]:
try:
     instances = aws_conn.describe_db_instances(DBInstanceIdentifier=rdsIdentifier)
     RDS_MYSQL_F_HOST = instances.get('DBInstances')[0].get('Endpoint').get('Address')
     print(RDS_MYSQL_F_HOST)
except Exception as ex:
     print("La instancia de base de datos no existe o aun no se ha terminado de crear.")
     print(ex)

fuenteingresosmunimysql.c1dhtoyskhfb.us-east-2.rds.amazonaws.com


#### Conexión y creación de BD desde Python

In [15]:
import sql_query_fuente

try:
    myDw = mysqlC.connect(
    host=RDS_MYSQL_F_HOST, 
    user=config.get('RDS_MYSQL_F', 'DB_USER'),
    password=config.get('RDS_MYSQL_F', 'DB_PASSWORD'),
    database=config.get('RDS_MYSQL_F', 'DB_NAME')
    )

    mycursor = myDw.cursor()
    mycursor.execute(sql_query_fuente.DDL_QUERY, multi=True)
    myDw.commit()
    print("Data Warehouse Creado Exitosamente")
except Exception as ex:
    print("ERROR: Error al crear la base de datos.")
    print(ex)

Data Warehouse Creado Exitosamente
